In [26]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [27]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]

In [28]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [29]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [30]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [31]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [32]:
df_ma.dropna(inplace=True)

In [47]:
df_ma.reset_index(drop=True, inplace=True)

In [40]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
63,2022-06-09T04:00:00.000000000Z,1.07294,1.07330,1.07262,1.07266,1.072274,1.070062,0.002212,NaN,False
64,2022-06-09T05:00:00.000000000Z,1.07266,1.07296,1.07187,1.07190,1.072135,1.070069,0.002066,0.002212,False
65,2022-06-09T06:00:00.000000000Z,1.07188,1.07200,1.07090,1.07146,1.072029,1.070064,0.001964,0.002066,False
66,2022-06-09T07:00:00.000000000Z,1.07148,1.07168,1.07018,1.07022,1.071781,1.070072,0.001710,0.001964,False
67,2022-06-09T08:00:00.000000000Z,1.07022,1.07178,1.06940,1.07155,1.071700,1.070095,0.001605,0.001710,False


In [34]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

In [36]:
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)

In [38]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [48]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

In [52]:
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [53]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
14,2022-06-09T18:00:00.000000000Z,1.06388,1.06438,1.06262,1.06264,1.069483,1.069952,-0.000469,0.000056,True
103,2022-06-15T11:00:00.000000000Z,1.04824,1.04893,1.04552,1.04582,1.045129,1.044815,0.000315,-0.000077,True
117,2022-06-16T01:00:00.000000000Z,1.04528,1.04590,1.04312,1.04344,1.043273,1.043489,-0.000216,0.000117,True
124,2022-06-16T08:00:00.000000000Z,1.03969,1.04129,1.03840,1.04095,1.043343,1.043271,0.000071,-0.000145,True
127,2022-06-16T11:00:00.000000000Z,1.04051,1.04264,1.03836,1.03858,1.042839,1.043139,-0.000301,0.000089,True


In [54]:
df_plot = df_ma.iloc[0:26].copy()

In [45]:
df_plot.shape

(50, 10)

In [55]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(
        x=df_plot.time,
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
                  margin=dict(l=10,b=10,t=10),
                  font=dict(size=10,color="#e1e1e1"),
                  paper_bgcolor="#1e1e1e",
                  plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
)
fig.show()